In [8]:
import praw
import pandas as pd
from datetime import datetime
from praw.models import MoreComments
import preprocessor as pre
import regex as re

# TODO  
- update script to work with praw.ini configuration 
- fit infleuncer scores


# Reddit Instance

In [9]:
# Creates Reddit user instance 
# Connects to Reddit using Wisdm account


reddit = praw.Reddit(
    client_id="auD_kIwyQ1r3hfxTQEYuGw",
    client_secret="XyLGeMB1mJqoaB0lCXrk4Jtmy515AA",
    password="wisdmai1234",
    user_agent="wisdm",
    username="Dramatic-Ad-9651",
    check_for_async=False
)

In [10]:
# make sure we're connected to the api
print(reddit.user.me())

Dramatic-Ad-9651


In [11]:
#tickers 
tickers = list(set(['SPY','TSLA','PYPL', 'GME', 'VIX', 
                    'AMD', 'F', 'BBBY', 'NFLX', 'NVDA', 
                    'AAPL', 'INTC', 'FSR', 'QQQ', 'TLRY',
                     'MSFT', 'TWTR', 'SNAP', 'HOOD', 'WMT', 
                     'PTON', 'WISH', 'CPRX', 'AMC', 'SNDL', 
                     'AMZN', 'DIS', 'NIO', 'FB', 'NFLX', 'LCID', 'NVDA']))

tickers[0:5]


['SNDL', 'TSLA', 'GME', 'DIS', 'AMZN']

In [12]:
subreddits = [ 'wallstreetbets', 'stocks',  'investing', 'finance']
subreddits

['wallstreetbets', 'stocks', 'investing', 'finance']

# Subreddit

In [13]:
for thread in subreddits:
    subreddit = reddit.subreddit(thread)
    print(subreddit)

wallstreetbets
stocks
investing
finance


In [14]:
wsb = reddit.subreddit('wallstreetbets')
wsb

Subreddit(display_name='wallstreetbets')

In [15]:
for post in wsb.search('gme', sort = 'top', time_filter = 'day', limit = 10):
    print(post.title)

…And it’s gone. I set a new personal high score last Friday, only for it all to disappear in a week. I’ll see you guys again when I save up a couple thousand dollars. I will be back! But I’m sticking to SPY weeklies next time. Meme stock weeklies have brought me nothing but pain.
I think I’m almost there
Humble GME YOLO Update part two: "Dearly Regarded."
WSB banbet lost a week ago
GameStop - GME Stock Evaluation


In [16]:
for post in wsb.search('gme', sort = 'hot', time_filter = 'day', limit = 10):
    print(post.title)

Humble GME YOLO Update part two: "Dearly Regarded."
GameStop - GME Stock Evaluation
…And it’s gone. I set a new personal high score last Friday, only for it all to disappear in a week. I’ll see you guys again when I save up a couple thousand dollars. I will be back! But I’m sticking to SPY weeklies next time. Meme stock weeklies have brought me nothing but pain.
I think I’m almost there
WSB banbet lost a week ago


* need to specify which to sort by: hot, top, or mew 
* filter on day 

# Version 1

In [17]:
posts = pd.DataFrame()

ticker = 'GME'

for post in wsb.search(ticker.lower(), sort = 'new', time_filter = 'day', limit = None):
    #check if title has stock ticker 
    if ticker.lower() not in post.title.lower(): 
        #print(post.title)
        continue 
    #check if author is not banned 
    if hasattr(post.author, 'is_suspended'):
        #print(post.author.is_suspended)
        continue

    #collect desired values 
    title_instance = {
        'TYPE': 'TITLE',
        'ticker': ticker, 
        'subreddit': 'subreddit',
        'post_title': post.title, 
        'content': post.title, 
        'upvotes': post.score, 
        #'upvote_ratio': post.upvote_ratio,
        'num_comments': post.num_comments, 
        'author_karma': post.author.comment_karma, 
        'author_verified': post.author.has_verified_email, 
        'time': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
    }

    #create row and concat it to the df
    row = pd.DataFrame([title_instance])
    posts = pd.concat([posts, row], axis = 0, ignore_index = True)

    #checking comments
    for comment in post.comments:
        #do not want sub comments of comments 
        if isinstance(comment, MoreComments):
            continue
        if 'user report' in comment.body.lower(): 
            continue 
        comment_instance = {
            'TYPE': 'COMMENT',
            'ticker': ticker, 
            'subreddit': 'subreddit', 
            'post_title': post.title,
            'content': comment.body, 
            'upvotes': comment.score, 
            #'upvote_ratio': post.upvote_ratio,
            'num_comments': comment.replies.__len__(), 
            'author_karma': comment.author.comment_karma, 
            'author_verified': comment.author.has_verified_email, 
            'time': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        }

        row = pd.DataFrame([comment_instance])
        posts = pd.concat([posts, row], axis = 0, ignore_index = True)



In [18]:
posts.head()

,TYPE,ticker,subreddit,post_title,content,upvotes,num_comments,author_karma,author_verified,time
0,TITLE,GME,subreddit,GameStop - GME Stock Evaluation,GameStop - GME Stock Evaluation,0,6,37372,True,2022-11-04
1,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,>I think you're wrong about the fundamentals. ...,14,0,296739,False,2022-11-04
2,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,gtfoh,9,1,8203,True,2022-11-04
3,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,Seems optimistic.,-1,0,20333,True,2022-11-04
4,COMMENT,GME,subreddit,GameStop - GME Stock Evaluation,This ain't no movement you fuckin mouthbreathe...,0,0,1000,True,2022-11-04


# Version 2

In [19]:
posts = pd.DataFrame()
comments = pd.DataFrame()

ticker = 'GME'

for post in wsb.search(ticker.lower(), sort = 'new', time_filter = 'day', limit = None):
    #check if title has stock ticker 
    if ticker.lower() not in post.title.lower(): 
        #print(post.title)
        continue 
    #check if author is not banned 
    if hasattr(post.author, 'is_suspended'):
        #print(post.author.is_suspended)
        continue

    #collect desired values 
    title_instance = {
        'ticker': ticker, 
        'subreddit': 'subreddit',
        'content': post.title, 
        'upvotes': post.score, 
        'upvote_ratio': post.upvote_ratio,
        'num_comments': post.num_comments, 
        'author_karma': post.author.comment_karma, 
        'author_verified': post.author.has_verified_email, 
        'time': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
    }

    #create row and concat it to the df
    row = pd.DataFrame([title_instance])
    posts = pd.concat([posts, row], axis = 0, ignore_index = True)

    #checking comments
    for comment in post.comments:
        #do not want sub comments of comments 
        if isinstance(comment, MoreComments):
            continue
        #remove user reports
        if 'user report' in comment.body.lower(): 
            continue
         
        
        comment_instance = {
            'ticker': ticker, 
            'subreddit': 'subreddit', 
            # optional can remove if no grouping by title is needed 
            'post_title': post.title,
            'content': comment.body, 
            'upvotes': comment.score, 
            'replies': comment.replies.__len__(), 
            'sticked': comment.stickied,
            'author_karma': comment.author.comment_karma, 
            'author_verified': comment.author.has_verified_email, 
            'time': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d'),
        }

        row = pd.DataFrame([comment_instance])
        comments = pd.concat([comments, row], axis = 0, ignore_index = True)

In [20]:
posts.head()


,ticker,subreddit,content,upvotes,upvote_ratio,num_comments,author_karma,author_verified,time
0,GME,subreddit,GameStop - GME Stock Evaluation,0,0.27,6,37372,True,2022-11-04
1,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...",9,0.72,8,47883,True,2022-11-04


In [60]:
comments.head()

,ticker,subreddit,post_title,content,upvotes,replies,sticked,author_karma,author_verified,time,clean_content
0,GME,subreddit,GameStop - GME Stock Evaluation,>I think you're wrong about the fundamentals. ...,14,0,False,296739,False,2022-11-04,i think youre wrong about the fundamentals the...
1,GME,subreddit,GameStop - GME Stock Evaluation,gtfoh,9,1,False,8203,True,2022-11-04,gtfoh
2,GME,subreddit,GameStop - GME Stock Evaluation,Seems optimistic.,-1,0,False,20333,True,2022-11-04,seems optimistic
3,GME,subreddit,GameStop - GME Stock Evaluation,This ain't no movement you fuckin mouthbreathe...,0,0,False,1000,True,2022-11-04,this aint no movement you fuckin mouthbreather...
4,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...",https://i.redd.it/mx76ovirckw91.png\n\nOrigina...,5,0,False,47883,True,2022-11-04,httpsiredditmxovirckwpng\n\noriginal position ...


# Preprocessing 

In [233]:
# function to clean text 
def preprocess_text(text):
    #remove url, emoji, smiley, number 
    # need to get module to work 
    #(pre.OPT.URL, pre.OPT.EMOJI, pre.OPT.smiley, pre.OPT.NUMBER)
    #cleantweet = pre.clean(text)

    #remove urls 
    cleantext = re.sub(r'(http|https|ftp|ftps)\:\/\/[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(\/\S*)?', '', fr"{text}")
    

    cleantext = re.sub('(\\n\\n)', ' ', cleantext)
    
    #convert to lowercase 
    cleantext = cleantext.lower() 


    #remove digits 
    cleantext = re.sub('\d+', '', fr"{cleantext}")
    
    #remove non word characters 
    cleantext = re.sub(r'[^\w\s]', '', fr"{cleantext}")

    return cleantext







In [234]:
posts['clean_content'] = posts['content'].apply(preprocess_text)
comments['clean_content'] = comments['content'].apply(preprocess_text)

In [235]:
posts[['content', 'clean_content']].head()


,content,clean_content
0,GameStop - GME Stock Evaluation,gamestop gme stock evaluation
1,"Humble GME YOLO Update part two: ""Dearly Regar...",humble gme yolo update part two dearly regarded


In [236]:
comments[['content', 'clean_content']]

,content,clean_content
0,>I think you're wrong about the fundamentals. ...,i think youre wrong about the fundamentals the...
1,gtfoh,gtfoh
2,Seems optimistic.,seems optimistic
3,This ain't no movement you fuckin mouthbreathe...,this aint no movement you fuckin mouthbreather...
4,https://i.redd.it/mx76ovirckw91.png\n\nOrigina...,original position instead of being down im s...
5,>I am pleased to see that you are doing well w...,i am pleased to see that you are doing well wi...
6,"Hey, if you haven't already, please reply to t...",hey if you havent already please reply to this...
7,What's with the shitty robinhood screenshots? ...,whats with the shitty robinhood screenshots li...


In [237]:
posts.head()

,ticker,subreddit,content,upvotes,upvote_ratio,num_comments,author_karma,author_verified,time,clean_content
0,GME,subreddit,GameStop - GME Stock Evaluation,0,0.27,6,37372,True,2022-11-04,gamestop gme stock evaluation
1,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...",9,0.72,8,47883,True,2022-11-04,humble gme yolo update part two dearly regarded


In [238]:
comments.head()

,ticker,subreddit,post_title,content,upvotes,replies,sticked,author_karma,author_verified,time,clean_content
0,GME,subreddit,GameStop - GME Stock Evaluation,>I think you're wrong about the fundamentals. ...,14,0,False,296739,False,2022-11-04,i think youre wrong about the fundamentals the...
1,GME,subreddit,GameStop - GME Stock Evaluation,gtfoh,9,1,False,8203,True,2022-11-04,gtfoh
2,GME,subreddit,GameStop - GME Stock Evaluation,Seems optimistic.,-1,0,False,20333,True,2022-11-04,seems optimistic
3,GME,subreddit,GameStop - GME Stock Evaluation,This ain't no movement you fuckin mouthbreathe...,0,0,False,1000,True,2022-11-04,this aint no movement you fuckin mouthbreather...
4,GME,subreddit,"Humble GME YOLO Update part two: ""Dearly Regar...",https://i.redd.it/mx76ovirckw91.png\n\nOrigina...,5,0,False,47883,True,2022-11-04,original position instead of being down im s...


# Influencer Score